#### 4-5-1. 히스토그램 계산과 표시
 - 히스토그램: 전체 영상에서 각 픽셀값(0~255)의 개수를 파악하는데 유용함.
 - 전체 영상에서 픽셀들의 색상이나 명암의 분포를 파악하기 쉬움.
 - cv2.calcHist(img, channel, mask, histSize, ranges) 함수
  * img: 입력 영상. [img]처럼 리스트로 감싸서 표현.
  * channel: 처리할 채널, 리스트로 감싸서 표현.
    * 1채널: [0], 2채널: [0, 1], 3채널: [0, 1, 2]
  * mask: 마스크에 지정한 픽셀만 히스토그램 계산
  * histSize: 계급(bin)의 개수, 채널 개수에 맞게 리스트로 표현
    * 1채널: [256], 2채널: [256, 256], 3채널: [256, 256, 256]
  * ranges: 각 픽셀이 가질 수 있는 값의 범위, RGB인 경우 [0, 256]

In [ ]:
#practice. 그레이 스케일로 1채널 히스토그램
import cv2
import numpy as np
import matplotlib.pylab as plt

#이미지를 그레이 스케일로 읽기 및 출력하기 -- ①
img = cv2.imread('../img/mountain.jpg', cv2.IMREAD_GRAYSCALE)
cv2.imshow('img', img)

#히스토그램 계산 및 그리기 -- ②
hist = cv2.calcHist([img], [0], None, [256], [0, 256]) #대상 이미지(img), 1채널([0]), 마스크 None,  x축 개수([256]), 픽셀 최대~최소[0, 256]
plt.plot(hist)

print(hist.shape)            #히스토그램의 shape(256, 1) -- ③ 
print(hist.sum(), img.shape) #히스토그램의 총 합계의 이미지의 크기 -- ④
plt.show()

In [ ]:
#practice. 컬러 히스토그램
import cv2
import numpy as np
import matplotlib.pylab as plt

#이미지 읽기 및 출력
img = cv2.imread('../img/mountain.jpg')
cv2.imshow('img', img)

#히스토그램 계산 및 그리기
#컬러스케일 이미지의 3개 채널에 대해 1차원 히스토그램을 각각 구하고 나서 하나의 플롯에 그림.
#영역 순위: 파란색(하늘) > 초록색(나무) or 빨간색(단풍)
channels = cv2.split(img)
colors = ('b', 'g', 'r')
for (ch, color) in zip(channels, colors):
    hist = cv2.calcHist([ch], [0], None, [256], [0, 256])
    plt.plot(hist, color = color)
plt.show()

#### 4-5-2. 노멀라이즈(정규화)
 - 서로 기준이 다른 것을 하나의 절대적인 기준으로 만들어주기.
 - 절대적인 기준 대신 특정 구간으로 노멀라이즈하면 특정 부분에 몰려있는 값을 전체 영역으로 골고루 분포시킬 수도 있음.
  * ex. 점수 분포가 95, 96, 98, 98, 100이고 95점 이상에게 A+ 학점을 준다면 모두가 A+이 되어버림.
  * 이를 방지하기 위해 점수를 70~100점으로 다시 환산하려 할 때 구간 노멀라이즈가 필요함.
  * 새로운 구간의 차이(100 - 70)를 원래 구간의 차이(100 - 95)로 나눠주면, 원래 점수 1점의 차이는 새로운 점수로는 6점 차이라는 것을 알 수 있음.
  * 원래 점수가 5점 구간에서 얼마인지 찾아 그 비율과 곱해서 새로운 시작 구간에 더하면 새로운 점수를 구할 수 있다.
  
                            
        I(N) = {(I - Min) * (newMax - newMin) / (Max - Min)} + newMin

         * I: 노멀라이즈 이전 값
         * Min, Max: 노멀라이즈 이전 범위의 최소값, 최대값
         * newMin, newMax: 노멀라이즈 이후 범위의 최소값, 최대값
         * I(N): 노멀라이즈 이후 값
         
   * [95, 96, 97, 98, 99, 100] --> [70, 76, 72, 88, 94, 100]

 - dst = cv2.normalize(src, dst, alpha, beta, type_flag)
    * src = 노멀라이즈 이전 데이터
    * dst = 노멀라이즈 이후 데이터
    * alpha: 노멀라이즈 구간 1
    * beta: 노멀라이즈 구간 2, 구간 노멀라이즈가 아닌 경우 사용 안 함
    * type_flag: 알고리즘 선택 플래그 상수
      * cv2.NORM_MINMAX: 알파, 베타 구간으로 노멀라이즈
      * cv2.NORM_L1: 전체 합으로 나누기, alpha = 노멀라이즈 전체 합
      * cv2.NORM_L2: 단위 벡터(unit vector)로 노멀라이즈
      * cv2.NORM_INF: 최대값으로 나누기

In [ ]:
#practice.히스토그램 정규화
import cv2
import numpy as np
import matplotlib.pylab as plt

#그레이스케일로 영상 읽기 -- ①
img = cv2.imread('../img/abnormal.jpg', cv2.IMREAD_GRAYSCALE)

#직접 노멀라이즈 공식을 대입하여 연산한 정규화 -- ②
img_f = img.astype(np.float32)  
img_norm = ((img_f - img_f.min()) * 255 / (img_f.max() - img_f.min()))
img_norm = img_norm.astype(np.uint8)  #dtype 변환이 2번(float->uint8)하는 이유는 연산 과정에서 소수점이 발생해서임.

#OpenCV API를 이용한 정규화 -- ③
img_norm2 = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX)  #구간 대상값: alpha=0, beta = 255

#히스토그램 -- ④
hist = cv2.calcHist([img], [0], None, [256], [0, 255])
hist_norm = cv2.calcHist([img_norm], [0], None, [256], [0, 255])
hist_norm2 = cv2.calcHist([img_norm2], [0], None, [256], [0, 255])

cv2.imshow('Before', img)
cv2.imshow('Manaul', img_norm)
cv2.imshow('cv2.normalize()', img_norm2)   #중앙에 몰려있던 픽셀들의 분포가 전체적으로 고르게 퍼져 화질이 개선됨.

hists = {'Before' : hist, 'Manual' : hist_norm, 'cv2.normalize()': hist_norm2}
for i, (k, v) in enumerate(hists.items()):
    plt.subplot(1, 3, i+1)
    plt.title(k)
    plt.plot(v)
plt.show()

 - 서로 다른 히스토그램의 빈도를 같은 조건으로 비교하는 경우: 전체의 비율로 노멀라이즈.

          norm - cv2.normalize(hist, None, 1, 0, cv2.NORM_L1)

          * cv2.NORM_L1 상수 사용시 결과는 전체를 모두 합했을 때 1이 됨.
          * 세 번째 인자값에 따라 그 합은 달라지고, 네 번째 인자값은 무시된다.


#### 4-5-3.이퀄라이즈

 - normalize는 분포가 한 곳에 집중되어 있는 경우에는 효과적이나, 그 집중된 영역에서 멀리 떨어진 값이 있을 경우에는 효과가 없음.
 - ex. 5명의 점수가 70, 96, 98, 98, 100이면, 구간 노멀라이즈로는 70 ~ 100 분포로 만들어도 기존의 범위와 새로운 범위가 같기 때문에 결과는 동일하다.
 - 이때 필요한 것이 equalize(평탄화): 히스토그램으로 빈도를 구해서 -> 그 값을 normalize 한 후 -> 누적값을 전체 개수로 나눠 나온 결과값을 히스토그램 원래 픽셀 값에 매핑.
 
           H'(v) = round( (L-1) * { cdf(v) - (cdf_min) } / { (M * N) - (cdf_min) } )
           
           - cdf(v): 히스토그램 누적 함수
           - cdf_min: 누적 최소값, 1
           - M * N: 픽셀 수, 폭 * 높이
           - L: 분포 영역, 256
           - round(v): 반올림
           - H'(v): 이퀄라이즈된 히스토그램 값

 - Equalize는 명암 대비 개선에 효과적.
 - dst = cv2.equalizeHist(src[, dst])
    * src: 대상 이미지. 8비트 1채널.
    * dst: 결과 이미지.

In [ ]:
#practice: 그레이 스케일 이퀄라이즈 적용
import cv2
import numpy as np
import matplotlib.pylab as plt

#대상 영상을 그레이 스케일로 읽기 -- ①
img = cv2.imread("../img/yate.jpg", cv2.IMREAD_GRAYSCALE)
rows, cols = img.shape[:2]

#이퀄라이즈 연산을 직접 적용 -- ②
hist = cv2.calcHist([img], [0], None, [256], [0, 256])  #히스토그램 계산
cdf = hist.cumsum()                                     #누적 히스토그램
cdf_m = np.ma.masked_equal(cdf, 0)                      #0인 값을 NaN으로 제거(-> 불필요한 연산 줄임)
cdf_m = (cdf_m - cdf_m.min()) / (rows * cols) * 255     #이퀄라이즈 히스토그램 계산
cdf = np.ma.filled(cdf_m, 0).astype('uint8')            #NaN을 다시 0으로 환원
img2 = cdf[img]                                         #연산 결과인 히스토그램을 픽셀로 매핑

#OpenCV API로 이퀄라이즈 히스토그램 적용 -- ③
img3 = cv2.equalizeHist(img)

#이퀄라이즈 결과 히스토그램 계산
hist2 = cv2.calcHist([img2], [0], None, [256], [0, 256])
hist3 = cv2.calcHist([img3], [0], None, [256], [0, 256])

#결과 출력
cv2.imshow("Before", img)
cv2.imshow("Manual", img2)
cv2.imshow("cv2.equalizeHist()", img3)
hists = {'Before': hist, 'Manual': hist2, 'cv2.qeualizeHist()': hist3}
for i, (k, v) in enumerate(hists.items()):
    plt.subplot(1, 3, i+1)
    plt.title(k)
    plt.plot(v)
plt.show()   

In [ ]:
#practice. 컬러 이미지에 대한 이퀄라이즈 적용
import numpy as np
import cv2

img = cv2.imread('../img/yate.jpg')  #이미지 읽기, BGR 스케일

#컬러스케일 변경: BGR -> YUV -- ①
img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
#YUV 컬러스케일의 첫 번째 채널에 equalize 적용 -- ②
img_yuv[:, :, 0] = cv2.equalizeHist(img_yuv[:, :, 0])
#img_hsv[:, :, 2] = cv2.equlizeHist(img_yuv[:, :, 2]) ==> HSV 컬러 스페이스에 적용. 위와 비슷한 결과.

#컬러스케일 변경: YUV -> BGR
img2 = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)

cv2.imshow('Before', img)
cv2.imshow('After', img2)   #요트 부분은 훨씬 선명해짐.
cv2.waitKey()
cv2.destroyAllWindows()

#### 4-5-4.CLAHE

 - Contrast Limiting Adaptive Histogram Equalization: 영상 전체에 이퀄라이즈를 적용했을 때 너무 밝은 부분이 날아가는 현상을 막기 위해, 영상을 일정한 영역으로 나눠 이퀄라이즈를 적용.
 - 어느 히스토그램 계급(bin)이든 지정된 제한 값을 넘으면 그 픽셀은 다른 계급으로 배분하고 나서 이퀄라이즈를 적용함.

 - clahe = cv2.createCLAHE(clipLimit, tileGridSize): CLAHE 생성
    * clipLimit: Contrast 제한 경계값. 기본 40.0
    * tileGridSize: 영역 크기. 기본 8 x 8
    * clahe: 생성된 CLAHE 객체.
 - clahe.apply(src): CLAHE 적용
    * src: 입력 영상.

In [ ]:
#practice. CLAHE 적용
import cv2
import numpy as np
import matplotlib.pylab as plt

#이미지를 읽어서 YUV 컬러 스페이스로 변경 -- ①
img = cv2.imread("../img/bright.jpg")     #빛이 너무 많이 들어간 원본 사진.
img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

#밝기 채널에 대해서 이퀄라이즈 적용 -- ②
img_eq = img_yuv.copy()
img_eq[:, :, 0] = cv2.equalizeHist(img_eq[:, :, 0])
img_eq = cv2.cvtColor(img_eq, cv2.COLOR_YUV2BGR)                #이퀄라이즈 적용시 밝은 곳이 날아가버림.

#밝기 채널에 대해서 CLAHE 적용 -- ③
img_clahe = img_yuv.copy()
clahe = cv2.createCLAHE(clipLimit = 3.0, tileGridSize = (8, 8)) #CLAHE 생성 #clipLimit 기본값이 40이므로 3.0으로 변경.
img_clahe[:, :, 0] = clahe.apply(img_clahe[:, :, 0])            #CLAHE 적용
img_clahe = cv2.cvtColor(img_clahe, cv2.COLOR_YUV2BGR)

#결과 출력 -- ④
cv2.imshow('Before', img)
cv2.imshow('CLAHE', img_clahe)
cv2.imshow('equalizeHist', img_eq)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
#### 4-5-5. 2D 히스토그램
import cv2
import matplotlib.pylab as plt

plt.style.use('classic')                            #컬러 스타일을 1.x 스타일로 사용 -- ①
img = cv2.imread("../img/mountain.jpg")

plt.subplot(131)       # (x,y)좌표가 (15, 25)인 지점 부근의 빨간색 ==> 파란색이면서 초록색인 픽셀의 개수가 가장 많은 부분이라는 뜻.
hist = cv2.calcHist([img], [0, 1], None, [32, 32], [0, 256, 0, 256]) # -- ② 계급 수를 256으로 조밀하게 하면 색상이 너무 작게 나오므로 32로 표현.
p = plt.imshow(hist)                                                 # -- ③ 
plt.title('Blue and Green')                                          # -- ④ 
plt.colorbar(p)                                                      # -- ⑤ 각 색상에 대한 컬러 막대 표시. 빨간색으로 표시될수록 픽셀 개수가 많고, 파란색은 픽셀이 적은 것을 의미함.

plt.subplot(132)
hist = cv2.calcHist([img], [1, 2], None, [32, 32], [0, 256, 0, 256]) # -- ⑥
p = plt.imshow(hist)
plt.title('Green and Red')
plt.colorbar(p)

plt.subplot(133)
hist = cv2.calcHist([img], [0, 2], None, [32, 32], [0, 256, 0, 256]) # -- ⑦
p = plt.imshow(hist)
plt.title('Blue and Red')
plt.colorbar(p)

plt.show()             #2차원 히스토그램의 의미: x축이면서 y축인 픽셀의 분포를 알 수 있음(AND 연산과 같음)

#### 4-5-6. 역투영
 - 2차원 히스토그램, HSV 컬러 스페이스를 이용해 색상으로 특정 물체나 사물의 일부분을 배경에서 분리할 수 있음.
 - 관심영역의 H와 V값의 분포를 얻어낸 후 전체 영상에서 해당 분포의 픽셀만 찾아내는 원리.

In [ ]:
#practice. 마우스로 선택한 영역의 물체 배경 제거.
import cv2
import numpy as np
import matplotlib.pyplot as plt

win_name = 'back_projection'
img = cv2.imread('../img/pump_horse.jpg')
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
draw = img.copy()

#역투영된 결과를 마스킹해서 결과를 출력하는 공통함수 -- ⑤
def masking(bp, win_name):
    disc = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5)) # cv2.getStructuringElement: 마스크 표면을 부드럽게 해 주는 함수
    cv2.filter2D(bp, -1, disc, bp)     # cv2.filter2D: 마스크 표면을 부드럽게 해 주는 함수
    _, mask = cv2.threshold(bp, 1, 255, cv2.THRESH_BINARY)
    result = cv2.bitwise_and(img, img, mask = mask)
    cv2.imshow(win_name, result)
    
#직접 구현한 역투영 함수 -- ⑥
def backProject_manual(hist_roi):
    #전체 영상에 대한 H, S 히스토그램 연산 --⑦
    hist_img = cv2.calcHist([hsv_img], [0, 1], None, [180, 256], [0, 180, 0, 256])
    #선택 영역과 전체 영상에 대한 히스토그램 비율 계산 -- ⑧    
    #관심영역과 비슷한 색상분포를 갖는 히스토그램일수록 1에 가까워지고 반대는 0에 가까워지므로, 마스킹에 사용하기 좋음.
    hist_rate = hist_roi / (hist_img + 1)   # ①에서 전달된 관심영역의 히스토그램을 전체 영상의 히스토그램으로 나눔
                                            #1을 더하는 이유: 분모가 0이 되는 것을 방지하기 위해.
    #원래 영상의 H와 S 픽셀값에 hist_rate의 비율을 매핑 -- ⑨
    h, s, v = cv2.split(hsv_img)
    bp = hist_rate[h.ravel(), s.ravel()] #bp = H와 S가 교차되는 지점의 비율을 그 픽셀의 값으로 하는 1차원 배열. 
    bp = np.minimum(bp, 1)               #bp는 비율이므로 1을 넘어서는 안됨. 따라서 1을 넘는 수는 1을 갖게 함.
    bp = bp.reshape(hsv_img.shape[:2])   #1차원 배열을 원래의 shape로 변환.
    cv2.normalize(bp, bp, 0, 255, cv2.NORM_MINMAX)  #0~255의 그레이 스케일에 맞는 픽셀값으로 노멀라이즈.
    bp = bp.astype(np.uint8)             #변환 도중 float타입으로 변경된 원소를 uint8로 변경.
    #역투영 결과로 마스킹해서 결과 출력 -- ⑩
    masking(bp, 'result_manual')
    
#OpenCV API로 구현한 함수 -- ⑪
def backProject_cv(hist_roi):
    #역투영 함수 호출 -- ⑫
    bp = cv2.calcBackProject([hsv_img], [0, 1], hist_roi, [0, 180, 0, 256], 1)
    #역투영 결과로 마스킹해서 결과 출력 ⑬
    masking(bp, 'result_cv')
    
#ROI 선택 -- ①
(x, y, w, h) = cv2.selectROI(win_name, img, False)   # -- 마우스로 영역 선택
if w > 0 and h > 0:
    roi = draw[y:y+h, x:x+w]
    cv2.rectangle(draw, (x, y), (x+w, y+h), (0, 0, 255), 2)
    #선택한 ROI를 HSV 컬러스페이스로 변경 -- ②
    hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)   # -- ROI 선택 후 스페이스나 엔터 키 누르면 ② 실행 
    #H,S 채널에 대한 히스토그램 계산 -- ③              # 2개 채널이므로 2차원 히스토그램
    hist_roi = cv2.calcHist([hsv_roi], [0, 1], None, [180, 256], [0, 180, 0, 256])
    #ROI의 히스토그램을 매뉴얼 구현함수와 OpenCV를 이용하는 함수에 각각 전달 -- ④
    backProject_manual(hist_roi)
    backProject_cv(hist_roi)
    
    
cv2.imshow(win_name, draw)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
 # 위 셀 29번 라인,  bp = hist_rate[h.ravel(), s.ravel()] 을 단순화하면 아래와 같음.

v = np.arange(6).reshape(2, 3)
print(v)
row = np.array([1, 1, 1, 0, 0, 0])
col = np.array([0, 1, 2, 0, 1, 2])
print(v[row, col])

 - OpenCV의 역투영 함수  

         cv2.calcBackPrjoect(img, channel, hsot, ranges, scale)
             * img: 입력 영상. [img]처럼 리스트로 감싸서 표현.
             * channel: 처리할 채널, 리스트로 감싸서 표현.
               * 1채널: [0], 2채널: [0, 1], 3채널:[0, 1, 2]
             * hist: 역투영에 사용할 히스토그램
             * ranges: 각 픽셀이 가질 수 있는 값의 범위
             * scale: 결과에 적용할 배율 계수
 - 역투영의 장점: 알파 채널이나 크로마 키 같은 보조 역할이 없어도 복잡한 모양의 사물을 분리할 수 있음.
 - 역투영의 단점: 대상 사물의 색상과 비슷한 색상이 뒤섞여 있을 때는 효과가 떨어지는 단점이 있음.

#### 4-5-7. 히스토그램 비교

 - 히스토그램: 영상의 픽셀값의 분포를 갖는 정보
 - 히스토그램 비교를 통해 영상에 사용한 픽셀의 색상 비중이 얼마나 비슷한지 알 수 있음(= 영상의 유사도)

         cv2.compareHist(hist1, hist2, method)
            * hist1, hist2: 비교할 2개의 히스토그램. 크기와 차원이 같아야 함.
            * method: 비교 알고리즘 선택 플래그 상수(유사성 측정 척도가 다름)
                ** cv2.HISTCMP_CORREL: 상관관계(1: 완전 일치, -1: 최대 불일치, 0: 무관계). 
                                       피어슨 상관관계로 유사성 측정.
                ** cv2.HISTCMP_CHISQR: 카이제곱(0: 완전 일치, 큰 값(미정): 최대 불일치).
                ** cv2.HISTCMP_INTERSECT: 교차(1: 완전 일치, 0: 최대 불일치(1로 정규화한 경우))
                     두 히스토그램의 교차점의 작은 값을 선택해서 그 합을 반환.
                     반환값을 원래의 히스토그램의 합으로 나누면 1, 0으로 노멀라이즈 할 수 있음.
                ** cv2.HISTCMP_BHATTACHARYYA: 바타차야(0: 완전 일치, 1: 최대 불일치). 
                                              두 분포의 중첩되는 부분 측정.
                ** cv2.HISTCMP_HELLINGER: HISTCMP_BHATTACHARYYA와 동일.

In [ ]:
#practice. 히스토그램 비교.
import cv2
import numpy as np
import matplotlib.pylab as plt

img1 = cv2.imread('../img/taekwonv1.jpg')
img2 = cv2.imread('../img/taekwonv2.jpg')
img3 = cv2.imread('../img/taekwonv3.jpg')
img4 = cv2.imread('../img/dr_ochanomizu.jpg')

cv2.imshow('query', img1)
imgs = [img1, img2, img3, img4]
hists = []
for i, img in enumerate(imgs):
    plt.subplot(1, len(imgs), i+1)
    plt.axis('off')
    plt.imshow(img[:, :, ::-1])
    #각 이미지를 HSV로 변환 -- ①
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #H, S 채널에 대한 히스토그램 계산 -- ②
    hist = cv2.calcHist([hsv], [0, 1], None, [180, 256], [0, 180, 0, 256])
    #0~1로 정규화 -- ③
    cv2.normalize(hist, hist, 0, 1, cv2.NORM_MINMAX)
    hists.append(hist)
    
query = hists[0]
methods = {'CORREL': cv2.HISTCMP_CORREL,
           'CHISQR': cv2.HISTCMP_CHISQR,
           'INTERSECT': cv2.HISTCMP_INTERSECT,
           'BHATTACHARYYA': cv2.HISTCMP_BHATTACHARYYA}
for j, (name, flag) in enumerate(methods.items()):
    print('%-10s'%name, end = '\t')
    for i, (hist, img) in enumerate(zip(hists, imgs)):
        #각 메서드에 따라 img1과 각 이미지의 히스토그램 비교 -- ④
        ret = cv2.compareHist(query, hist, flag)
        if flag == cv2.HISTCMP_INTERSECT:    #교차분석인 경우:
            ret = ret/np.sum(query)          #비교대상으로 나누어 1로 정규화.
        print("img%d:%7.2f"%(i+1, ret), end = '\t')
plt.show

# ① ~ ③은 각 영상을 HSV 컬러 스페이스로 바꾸고, H와 V에 대해 2차원 히스토그램을 계산해서 0 ~ 1로 노멀라이즈.
# ④에서 비교 알고리즘을 이용해 각 영상을 차례대로 비교.
# cv2.HISTCMP_INTERSECT인 경우, 비교 원본의 히스토그램으로 나누기를 하면 0 ~ 1로 노멀라이즈 할 수 있고, 결과 판별이 편리함.
#img1의 비교 결과는 완전 일치, img4의 경우 가장 멀어진 값으로 나타남.